# Exercise 0.2 - pandas
prepared by M.Hauser

[pandas](https://pandas.pydata.org/) is a "Python package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, real world data analysis in Python. Additionally, it has the broader goal of becoming the most powerful and flexible open source data analysis / manipulation tool available in any language."

Thus, pandas provides R-like data structures, e.g. a `DataFrame`, and associated methods. As pandas is concentrating on the handling of 2D data, I'am not a heavy user of this library. However, some of it's functionality is very handy.

A more comprehensive introduction is given in [10 Minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html).

In [ ]:
# pandas is conventionally abbreviated as `pd`
import pandas as pd

import numpy as np

## Data types

### Series

Series are 1-dimensional, and each element is labeled, in our example with 'a', 'b', .... The axis labels are collectively referred to as the index. 


In [ ]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])

s

### DataFrame

DataFrame is the most common data type of pandas. It is 2-dimensional, has an index and (potentially) several colums.

In [ ]:
dates = pd.date_range('2013-01-28', periods=6)

df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))

df

## Selection

There are various ways to select data, see the [introduction](https://pandas.pydata.org/pandas-docs/stable/10min.html#selection).

In [ ]:
# select the column named 'A'

df['A']

# this yields a 'Series' object 

In [ ]:
# select the column named 'A'

df[['A']]

# this yields a 'DataFrame' object with only a single column.

In [ ]:
# select a single date

df.loc['2013-01-30', :]

In [ ]:
# select a single all days in a month

df.loc['2013-01', :]

In [ ]:
# select two columns

df.loc[:, ['A', 'B']]

In [ ]:
# if you want to select by position and not by label you have to use .iloc

df.iloc[0:3, [0, 1]]

In [ ]:
# item assignment can also be done

df.iloc[3, 1] = np.NaN

df.loc['2013-02', 'D'] = np.NaN

df

## Computation



In [ ]:
# it's easy to compute statistics of the data

df.mean()

Missing values (`NaN`) are generally left out. Unless explicitely included:

In [ ]:
df.mean(skipna=False)

In [ ]:
# the mean can also be taken over the columns

df.mean(axis=1)

### Exercises

 * What happens if we add a column named 'E', and only assign a value for the first day? (Hint: `df.loc['2013-01-28', 'E'] = 5`)
 * Add a new column named 'F' with where all values are 2.
 * From what you know, calculate the standard deviation of the values in February, for all columns.

In [ ]:
# all the other rows are NaN
# f.loc[:, 'F'] = 2
# df.loc['2013-02' , :].std()

## Reading comma seperated values (csv) from a file

For the rest of the exercise we use measurements of the atmospheric CO<sub>2</sub> concentration from Mauna Loa, Hawaii (Keeling & Whorf 2004), see how the [data was prepared](./../data/prepare_CO2_mauna_loa.ipynb).

The CO$_2$ data is saved in a .csv file:

In [ ]:
# we first take a look at the data

! head ./../data/co2.csv

# "!" executes a system command

In [ ]:
# alternative way to print the head of the file

fN = './../data/co2.csv'

with open(fN) as fid:
    for i in range(10):
        print(fid.readline(), end='')


pandas has a powerful function to read from .csv files, `pd.read_csv`, let's display the documentation:

In [ ]:
pd.read_csv?

Althought the defaults are quite good, it usually requires some tweaking... (I always need to check the documentation...). 

For our file we need to pass at least the filename (fN) and the number of the header row, (which gives the name for the column, here 'co2'). Because python is zero-based this is line number 6 - 1 = 5, thus, we pass `header=5`.

In [ ]:
co2_prelim = pd.read_csv(fN, header=5)

co2_prelim.head(3)

In [ ]:
# let's check how the date format is saved

print('The first date', co2_prelim.iloc[0, 0])
print('The type of the date', type(co2_prelim.iloc[0, 0]))

This is not entirely what we want, yet.

 * We want the labels to be the (time) axis, and not numbers. Therefore we have to choose `index_col=0`, which uses the first column as index.
 * We also want the time axis to be a pandas [`DateTimeIndex`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.html), and not a string. Thus, we use `parse_dates=True`.
 
See `pd.read_csv?` for details.

In [ ]:
co2 = pd.read_csv(fN, index_col=0, parse_dates=True, header=5)

co2.head()

In [ ]:
co2.index

## Time Series

pandas has very powerful [time series functionality](https://pandas.pydata.org/pandas-docs/stable/10min.html#time-series). We already saw the selection of months by using `df.loc['2013-02', :]`. `resample` is another, very helpful function. We can use it to convert the weekly CO$_2$ time series to a monthly time series:

In [ ]:
co2_monthly = co2.resample('M').mean()

co2_monthly.head()

Another time series functionality I often use, is [`date_range`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.date_range.html). It can be used to construct `DatetimeIndex` with a fixed frequency.

In [ ]:
# every day of the year 2000
pd.date_range('2000-01-01', '2000-12-31')

In [ ]:
# alternatively
pd.date_range('2000-01-01', periods=365)

In [ ]:
# every month of the year 2000
pd.date_range('2000-01-01', '2000-12-31', freq='M')

In [ ]:
# every second month of the year 2000
pd.date_range('2000-01-01', '2000-12-31', freq='2M')

### Exercises

* Calculate annual mean CO$_2$. (Hint: [offset aliases](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases))
* Create a `DatetimeIndex` from 01.05.2000 to 31.05.2000 with a half-hourly frequency.
* Calculate the standard deviation of `df` for each month again, this time using `resample`.

In [ ]:
# co2.resample('A').mean()
# pd.date_range('2000-05-01', '2000-05-31', freq='30min')
# df.resample('M').std()